---
title: "Trade Envy Report"
date: last-modified
engine: jupyter
execute:
  echo: false
  enabled: true
---

In [1]:
import plotly.graph_objects as go
import psycopg2
import pandas as pd
from cow_amm_trade_envy.models import Pools
import os
from dotenv import load_dotenv

load_dotenv()

DB_PARAMS = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
}


def fetch_data_from_postgres(query: str, params: tuple = ()):
    """Fetch data from a Postgres database."""
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cursor:
            cursor.execute(query, params)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
    return pd.DataFrame(data, columns=columns)


def analyze_surplus(network):
    table_name = f"{network}_envy"
    query = f"SELECT * FROM trade_envy.{table_name} WHERE trade_envy > 0"
    df = fetch_data_from_postgres(query)

    # Group by pool and calculate unused and used surplus
    grouped = df.groupby("pool").apply(
        lambda g: {
            "unused_surplus": g[~g["is_used"]]["trade_envy"].sum(),
            "used_surplus": g[g["is_used"]]["trade_envy"].sum(),
        },
        include_groups=False,
    )

    results = grouped.reset_index().rename(columns={0: "surplus"})
    results["unused_surplus"] = results["surplus"].apply(lambda x: x["unused_surplus"])
    results["used_surplus"] = results["surplus"].apply(lambda x: x["used_surplus"])
    notna_pools = results["pool"] != "None"
    results.loc[notna_pools, "pool"] = results.loc[notna_pools, "pool"].apply(
        lambda x: Pools().get_name_from_address(x)
    )
    return results


# Analyze surplus for a given network
network = "ethereum"
surplus_data = analyze_surplus(network)

# Prepare data for the stacked bar plot
pools = surplus_data["pool"]
unused_surplus = surplus_data["unused_surplus"]
used_surplus = surplus_data["used_surplus"]

# Create a stacked bar plot
fig = go.Figure()

# Add unused surplus (bottom of the stack)
fig.add_trace(
    go.Bar(
        name="Unused Surplus",
        x=pools,
        y=unused_surplus,
        marker_color="green",
    )
)

# Add used surplus (top of the stack)
fig.add_trace(
    go.Bar(
        name="Used Surplus",
        x=pools,
        y=used_surplus,
        marker_color="orange",
    )
)

# Update layout for better visualization
fig.update_layout(
    title="Comparison of Surplus Usage by Pool",
    xaxis_title="Pools",
    yaxis_title="Surplus Amount",
    template="plotly",
    barmode="stack",  # Make the bars stacked
)

# Show the plot
fig.show()